In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
#     "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
#     "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
#     "unsloth/llama-3-8b-Instruct-bnb-4bit",
#     "unsloth/llama-3-70b-bnb-4bit",
#     "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
#     "unsloth/Phi-3-medium-4k-instruct",
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    #target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from datasets import load_dataset, concatenate_datasets,Dataset
import pandas as pd

# 加载本地数据集
#dataset = load_dataset('json', data_files='/home/jovyan/train.jsonl', split='train')
# dataset = load_dataset("vicgalle/alpaca-gpt4", split = "train")
dataset1 = load_dataset('json', data_files='/home/jovyan/common_en_70k.jsonl', split='train')
print(len(dataset1))
dataset2 = load_dataset('json', data_files='/home/jovyan/common_zh_70k.jsonl', split='train')
print(len(dataset2))
dataset3 = load_dataset('json', data_files='/home/jovyan/computer_cn_26k_continue.jsonl', split='train')
print(len(dataset3))
dataset4 = load_dataset('json', data_files='/home/jovyan/computer_en_26k_continue.jsonl', split='train')
print(len(dataset4))
dataset6 = load_dataset('json', data_files='/home/jovyan/computer_en_26k(fixed).jsonl', split='train')
print(len(dataset6))
dataset5 = load_dataset('json', data_files='/home/jovyan/computer_zh_26k(fixed).jsonl', split='train')
print(len(dataset5))
dataset7 = load_dataset('json', data_files='/home/jovyan/unknow_zh_38k.jsonl', split='train')
print(len(dataset7))
dataset8 = load_dataset('json', data_files='/home/jovyan/unknow_zh_38k_continue.jsonl', split='train')
# 定义转换函数，将每个 conversation 转换为指定格式
combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3,dataset4, dataset5,dataset6,dataset7, dataset8])

# 定义转换函数，将每个 conversation 转换为指定格式
def convert_conversation_format(conversation):
    new_format = {'conversations': []}

    # 遍历每个对话，将 human 和 assistant 成对地插入
    for convo in conversation:
        new_format['conversations'].append({'content': convo['human'], 'role': 'user'})
        new_format['conversations'].append({'content': convo['assistant'], 'role': 'assistant'})

    return new_format
# 遍历 dataset 并进行转换
converted_data = [convert_conversation_format(item['conversation']) for item in combined_dataset]

# 打印转换后的第一个结果

# Convert the list of converted data into a DataFrame
df = pd.DataFrame(converted_data)

# Create a Dataset from the DataFrame
converted_data_dataset = Dataset.from_pandas(df)

# Verify the dataset
print(converted_data_dataset[0])

79107
76399
379
351
20692
20443
38557
{'conversations': [{'content': 'How much should I charge for web design by hour with profit if I need to make $8000 a month', 'role': 'user'}, {'content': "The hourly rate for web design can vary depending on your location, experience, and the type of projects you take on. However, if you need to make $8,000 a month, you would need to charge at least $160 per hour to reach that goal (assuming you work a standard 40-hour workweek). This rate would need to be adjusted based on your costs, expenses, and desired profit margin. It's important to keep in mind that this calculation is based on a standard 40 hour workweek, if you have another working schedule it may not apply.", 'role': 'assistant'}, {'content': 'What about if I spend 20hrs a week per project?', 'role': 'user'}, {'content': "If you spend 20 hours per week on a project, and you want to make $8,000 per month, you would need to charge a higher hourly rate to compensate for the fewer number of

In [ ]:
from datasets import load_dataset,concatenate_datasets
# 加载本地数据集
dataset = load_dataset('json', data_files='/home/jovyan/traintrain.jsonl', split='train')
## 数据集整合
from unsloth import to_sharegpt
dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\n{input}]]",
    output_column_name = "output",
    conversation_extension = 4, # Select more to handle longer conversations
)
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
for _ in range(11):
    dataset = concatenate_datasets([dataset, dataset])
print(len(dataset))
dataset = concatenate_datasets([dataset,converted_data_dataset])

63488


In [ ]:
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""
from unsloth import apply_chat_template
dataset = apply_chat_template(
    dataset,
    tokenizer = tokenizer,
    chat_template = chat_template,
    #default_system_message = "你是中企动力的客服",
)
print(dataset[0])
print(len(dataset))


Map:   0%|          | 0/301323 [00:00<?, ? examples/s]

{'conversations': [{'content': '中企动力和中企\n中企和中企动力是一家公司吗？', 'role': 'user'}, {'content': '‘中企动力”简称‘中企”，是一家成立于1999年的老牌互联网公司', 'role': 'assistant'}, {'content': '中企可以为客户制作以下门户网站\n中企能做什么类型的门户网站？', 'role': 'user'}, {'content': '内贸营销，全球营销，通用版本', 'role': 'assistant'}, {'content': '中企动力移动商城的定义\n移动商城能做什么？', 'role': 'user'}, {'content': '助力企业打造官方独立商城\n1、轻松实现私域锁客、老客复购、熟客裂变的全方位私域运营新生态;\n2、覆盖小程序+H5两端，助力企业开展私域营销，打造忠诚客户池，创造持续销量;', 'role': 'assistant'}, {'content': '中企动力的规模\n中企有多少员工？', 'role': 'user'}, {'content': '中企动力自成立起一直专注于企业与机构服务，总部位于北京，在全国24个省市拥有近70家分公司、5000名员工，累计服务150万+企业客户，其中规模以上企业客户80万+，涉及工业、食品、零售、农业、外贸等14个主流产业近百个细分行业，500强企业中有230+保持长期合作', 'role': 'assistant'}], 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nBelow are some instructions that describe some tasks. Write responses that appropriately complete each request.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n中企动力和中企\n中企和中企动力是一家公司吗？<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n‘中企动力”简称‘中企”，是一家成立

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 200,
        # num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)



# 查看资源使用
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

Map (num_proc=2):   0%|          | 0/301323 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


GPU = NVIDIA GeForce RTX 4070 SUPER. Max memory = 11.994 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 301,323 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.468300
2,2.075700
3,1.592200
4,1.646300
5,1.753600
6,1.481800
7,1.337500
8,1.136700
9,1.425200
10,1.398600


In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
import torch
torch.__version__
# 安装 ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

483.0925 seconds used for training.
8.05 minutes used for training.
Peak reserved memory = 9.34 GB.
Peak reserved memory for training = 3.356 GB.
Peak reserved memory % of max memory = 77.872 %.
Peak reserved memory for training % of max memory = 27.981 %.


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
messages = [                         # Change below!
    {"role": "user",      "content": "你好呀"}]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, streamer = text_streamer, max_new_tokens = 128, pad_token_id = tokenizer.eos_token_id)

你好！我是AI小助手，很高兴见到你！有什么问题吗？<|eot_id|>
